In [25]:
import pandas as pd
import requests
from io import StringIO

# %conda install lxml

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

url = "https://en.wikipedia.org/wiki/List_of_footballers_with_500_or_more_goals"

res = requests.get(url, headers=headers).text
res = StringIO(res)

df = pd.read_html(res)
df

[    0                                                  1
 0 NaN  This section needs additional citations for ve...,
    Rank              Player    Club                  Country and other  \
    Rank              Player  League  Cup Continental Country and other   
 0     1   Cristiano Ronaldo  590[a]   57         172               143   
 1     2       Lionel Messi*  553[b]   71         157               115   
 2     3               Pelé*  604[c]   49          26                83   
 3     4             Romário  545[d]   93          54                64   
 4     5       Ferenc Puskás  516[e]   69          56                84   
 5     6        Josef Bican*  515[f]  137          38                32   
 6     7  Robert Lewandowski  423[g]   62         117                88   
 7     8        Jimmy Jones*  330[h]  286          14                 9   
 8     9        Gerd Müller*  405[i]   92          69                68   
 9    10       Joe Bambrick*  347[j]  253           5     

In [26]:
df = df[3]

In [27]:
#df.to_csv("./data/top500goals.csv", index=False)

In [28]:
df = pd.read_csv("./data/top500goals.csv")
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989+,582,1.70,1924–1951
1,2,Cristiano Ronaldo,975,1337,0.73,2002–present
2,3,Josef Bican,950+,624,1.52,1930–1957
3,4,Ronnie Rooke,934+,1030,0.91,1929–1961
4,5,Lionel Messi,925,1194,0.77,2003–present


In [29]:
import random, math

df["Career span"] = df["Career span"].str.split("–").str[0]

In [30]:
df["Career span"].isna().sum()

np.int64(0)

In [31]:
#df.loc[~df["Goals"].str.isnumeric()]
df["Goals"] = [item.replace("+", "") for item in df["Goals"]]
df["Goals"]

0     989
1     975
2     950
3     934
4     925
     ... 
78    504
79    504
80    503
81    502
82    500
Name: Goals, Length: 83, dtype: object

In [32]:
df["Goals"] = df["Goals"].astype(int)
df["Career span"] = df["Career span"].astype(int)
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989,582,1.70,1924
1,2,Cristiano Ronaldo,975,1337,0.73,2002
2,3,Josef Bican,950,624,1.52,1930
3,4,Ronnie Rooke,934,1030,0.91,1929
4,5,Lionel Messi,925,1194,0.77,2003


In [33]:
df["Matches"] = df["Matches"].str.replace("+", "").astype(int)


df.describe()

,Rank,Goals,Matches,Ratio,Career span
count,83.000000,83.000000,83.000000,83.000000,83.00000
mean,42.000000,606.361446,724.915663,0.886024,1945.53012
std,24.103942,122.085394,193.411996,0.267563,26.39306
min,1.000000,500.000000,324.000000,0.530000,1891.00000
25%,21.500000,521.000000,590.500000,0.695000,1927.50000
50%,42.000000,564.000000,711.000000,0.860000,1938.00000
75%,62.500000,631.000000,836.500000,0.960000,1961.00000
max,83.000000,989.000000,1337.000000,1.700000,2010.00000


In [35]:
from datetime import datetime, timedelta

rows = []
columns = ["name","date", "goals", "assists"]

for row in df.itertuples():
    name = row.Player
    start_date = f"{row._6}-08-01"
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    goal_ratio = row.Ratio
    goal_sd = random.random() 
    goal_sd = goal_sd if goal_sd < 0.7 else 0.7
    assist_sd = 1 - goal_sd
    assist_ratio = 1 - goal_ratio
    
    for i in range(row.Matches):
        name = name
        match_date = timedelta(days=3 * i) + start_date
        goals = random.normalvariate(goal_ratio, goal_sd)
        assists = random.normalvariate(assist_ratio, assist_sd)
        goals = round(goals) if goals > 0 else 0
        assists = round(assists) if assists > 0 else 0
        row = [name, match_date, goals, assists]
        rows.append(row)
        
        
games_df = pd.DataFrame(data=rows, columns=columns)
games_df.head()
        

,name,date,goals,assists
0,Erwin Helmchen,1924-08-01,2,0
1,Erwin Helmchen,1924-08-04,2,0
2,Erwin Helmchen,1924-08-07,1,0
3,Erwin Helmchen,1924-08-10,2,0
4,Erwin Helmchen,1924-08-13,1,0


In [36]:
games_df.shape

(60168, 4)

In [37]:
games_df.sample(10)

,name,date,goals,assists
18100,Fernando Peyroteo,1940-02-02,2,0
54784,Raich Carter,1932-11-14,1,0
55183,Dennis Guy,1961-03-23,1,0
19765,Gyula Zsengellér,1933-05-10,1,0
16791,Eusébio,1964-07-14,1,0
22696,Uwe Seeler,1959-08-14,1,1
6106,Ferenc Puskás,1948-05-03,1,0
43248,Hughie Ferguson,1918-04-09,1,2
56225,Willy van der Kuijlen,1966-08-07,1,0
4848,Jimmy Jones,1944-03-31,1,1


In [39]:
fake_df = games_df.groupby("name").agg(
    matches=("name", "count"),
    goals=("goals", "sum"),
    assists=("assists", "sum"),
    g_ratio=("goals", "mean"),
    a_ratio=("assists", "mean"),
).reset_index().sort_values(by="goals", ascending=False).reset_index(drop=True)

fake_df.head(10)

,name,matches,goals,assists,g_ratio,a_ratio
0,Robert Lewandowski,1079,1079,539,1.000000,0.499537
1,Luis Suárez,1042,1027,635,0.985605,0.609405
2,Cristiano Ronaldo,1337,1014,496,0.758414,0.370980
3,Romário,1003,1001,449,0.998006,0.447657
4,Ronnie Rooke,1030,992,346,0.963107,0.335922
5,Erwin Helmchen,582,988,3,1.697595,0.005155
6,Lionel Messi,1194,939,221,0.786432,0.185092
7,Josef Bican,624,937,0,1.501603,0.000000
8,Roberto Dinamite,869,852,516,0.980437,0.593786
9,Pelé,851,850,309,0.998825,0.363102


In [40]:
for i in range(10):
    print(f"{i} df: {df.loc[i, ['Player', 'Goals']]} | {i} fake: {fake_df.loc[i,["name", "goals"]]}")

0 df: Player    Erwin Helmchen
Goals                989
Name: 0, dtype: object | 0 fake: name     Robert Lewandowski
goals                  1079
Name: 0, dtype: object
1 df: Player    Cristiano Ronaldo
Goals                   975
Name: 1, dtype: object | 1 fake: name     Luis Suárez
goals           1027
Name: 1, dtype: object
2 df: Player    Josef Bican
Goals             950
Name: 2, dtype: object | 2 fake: name     Cristiano Ronaldo
goals                 1014
Name: 2, dtype: object
3 df: Player    Ronnie Rooke
Goals              934
Name: 3, dtype: object | 3 fake: name     Romário
goals       1001
Name: 3, dtype: object
4 df: Player    Lionel Messi
Goals              925
Name: 4, dtype: object | 4 fake: name     Ronnie Rooke
goals             992
Name: 4, dtype: object
5 df: Player    Jimmy Jones
Goals             840
Name: 5, dtype: object | 5 fake: name     Erwin Helmchen
goals               988
Name: 5, dtype: object
6 df: Player    Ferenc Puskás
Goals               802
Name: 6, d